# YOLO Model Training

The YOLO model is the model that is used to predict a bounding box around worm's head.
In order for the model to make successful detections, the model must be trained on example images, with corresponding annotations (labels). 
The set of example images with their corresponding annotations is called a dataset.
In this notebook, the YOLO model is trained on a dataset obtained from RoboFlow.

RoboFlow is a website specifically designed for creating datasets for neural-network models, and YOLO among them.
We decided to use Reboflow because of the ease-of-use it provides in the creation of new datasets, and in the process of data annotation.

In [ ]:
# fix imports
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import ultralytics
from ultralytics import YOLO
import numpy as np

from wtracker.utils.gui_utils import UserPrompt
from wtracker.utils.path_utils import join_paths

In [ ]:
################################ User Input ################################

# the folder which contains the dataset, or into which the dataset will be downloaded
# if None, the user will be prompted to select a folder
dataset_folder = None

# path to the YOLO training configuration file.

# on the dataset page you can click on ’export dataset’ and then
# ’show download code’. You can copy the code snippet to the appropriate place in the notebook of step 6 to download the dataset to the
# computer and use it for training.
train_config_file = None

############################################################################

if dataset_folder is None:
    dataset_folder = UserPrompt.open_directory("Select Dataset Folder")

if train_config_file is None:
    train_config_file = UserPrompt.open_file(
        "Select YOLO Training Configuration File", file_types=[("YAML files", "*.yaml")]
    )

dataset_file = join_paths(dataset_folder, "data.yaml")

print(f"Dataset Folder: {dataset_folder}")
print(f"Training Config: {train_config_file}")

### Download the Dataset [Optional]

If the dataset is already on the system there is no need to do this step.

Otherwose, to download the dataset, copy the code produced by Roboflow to the cell below and run it to download the dataset.
Make sure to follow the instructions in the below cell.

In [ ]:
################################ User Input ################################

# Paste the code snippet you got from RoboFlow here
# To get the code snippet, go to the dataset page, click on 'Export Dataset' and select YOLOv8
# Then click on 'Show Download Code',  copy the code snippet and paste it here
# Make sure you use the latest version of the dataset.

# IMPORTANT: 
# Change the line: 
# `dataset = version.download("yolov8")`
# to the line:
# dataset = version.download("yolov8", location=dataset_folder, overwrite=True)`

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY_HERE")
project = rf.workspace("technion-institute-of-technology").project("celegan-head")
version = project.version(5)
dataset = version.download("yolov8", location=dataset_folder, overwrite=True)

############################################################################

### Load the model

The loaded model can be completely untrained YOLO weights, or can be a pre-trained model, which was previously trained.  
It is recommended to use pre-trained models if possible, since their training process is usually faster.

In [ ]:
################################ User Input ################################

# path to a trained model on the disk or a model configuration file
# more info is here: https://docs.ultralytics.com/modes/train/
# link to download non-trained model weights: https://docs.ultralytics.com/tasks/detect/#models
# if None, the user will be prompted to select a file
model_path = None

############################################################################

if model_path is None:
    model_path = UserPrompt.open_file("Select YOLO model", [("weights file", "*.pt"), ("model config file", "*.yaml")])

model = YOLO(model=model_path, task="detect")

**Train The Model**

In [ ]:
################################ User Input ################################

# GPU index for training. If No GPU is available, set device to None or "cpu"
device = None 

# If needed, make changes to the YOLO training config file.

############################################################################

results = model.train(data=dataset_file, cfg=train_config_file, device=device)

**Export to faster format**

Export potentially improves the performance of the trained model.
The exported model is platform specific, and is optimized for a specific architecture based on the export parameters.s

In [ ]:
model = YOLO(model=model_path)

out = model.export(
    format="onnx",  # Use 'engine' for TensorRT (GPU), 'onnx' for ONNX (CPU/GPU).
    imgsz=384,
    half=False,  # Enables FP16 (half-precision) quantization, reducing model size and potentially speeding up inference on supported hardware.
    dynamic=True,  # Allows dynamic input sizes for ONNX and TensorRT exports, enhancing flexibility in handling varying image dimensions
    simplify=True,  # Simplifies the model graph for ONNX exports, potentially improving performance and compatibility
    verbose=False,
    batch=32,  # max supported batch size of the exported model
    workspace=8,  # max memory usage during the export process in GB
)

**Deploy Model to Roboflow**

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8", model_path=model_path)